In [1]:
import pandas as pd
import requests
import json

In [2]:
def get_and_format(econ_series):
    api = '&api_key=6a6010bf274d634fd7463838a57f0887'
    params = 'file_type=json'
    url = 'https://api.stlouisfed.org/fred/series/observations?series_id='+econ_series+'&file_type=json'+api
    request = requests.get(url)
    content = request.content
    content_json = json.loads(content)
    #print(json.dumps(content_json, indent = 2))    #useful for debugging
    df = pd.DataFrame.from_records(content_json['observations'])
    df = df.loc[:,['date','value']]
    df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
    df = df.set_index('date').asfreq(freq = 'Q', method = 'ffill')
    return df

#Retrieve the data
UNRATE = get_and_format('UNRATE')
CPIAUCSL = get_and_format('CPIAUCSL')
FEDFUNDS = get_and_format('FEDFUNDS')
NASDAQCOM = get_and_format('NASDAQCOM')
#NASDAQCOM HAS "." VALUES that can't be there
NASDAQCOM = NASDAQCOM.replace(to_replace = '.', method = 'ffill')


RGDP = get_and_format('A939RX0Q048SBEA')
CRNT_ACCT = get_and_format('IEABC')
HOUSING = get_and_format('HOUST')
LABOR_FRC = get_and_format('CLF16OV')
MINIMUM_WAGE = get_and_format('FEDMINNFRWG')
YOUTH_UNEM = get_and_format('SLUEM1524ZSUSA')

#Combine the data
df = pd.merge(UNRATE, CPIAUCSL, on = 'date', how = 'left')
df = pd.merge(df, FEDFUNDS, on = 'date', how = 'left')
df = pd.merge(df, NASDAQCOM, on = 'date', how = 'left')
df = pd.merge(df, RGDP, on = 'date', how = 'left')
#df = pd.merge(df, CRNT_ACCT, on ='date', how = 'left')
df = pd.merge(df, HOUSING, on = 'date', how = 'left')
df = pd.merge(df, LABOR_FRC, on = 'date', how = 'left')
df = pd.merge(df, MINIMUM_WAGE, on = 'date', how = 'left')
#df = pd.merge(df, YOUTH_UNEM, on = 'date', how = 'left')
df.columns = ['UNRATE', 'CPIAUCSL', 'FEDFUNDS','NASDAQCOM', 'RGDP',  'HOUSING', 'LABOR_FRC', 'MINIMUM_WAGE']
df = df.fillna(method = 'ffill')
df.to_csv('US_UNEM_QT.csv')

In [3]:
df

,UNRATE,CPIAUCSL,FEDFUNDS,NASDAQCOM,RGDP,HOUSING,LABOR_FRC,MINIMUM_WAGE
date,,,,,,,,
1948-03-31,4.0,23.5,NaN,NaN,14316.0,NaN,60070,0.40
1948-06-30,3.6,24.15,NaN,NaN,14495.0,NaN,60957,0.40
1948-09-30,3.8,24.36,NaN,NaN,14515.0,NaN,60815,0.40
1948-12-31,4.0,24.05,NaN,NaN,14464.0,NaN,61169,0.40
1949-03-31,5.0,23.91,NaN,NaN,14202.0,NaN,61073,0.40
1949-06-30,6.2,23.92,NaN,NaN,14098.0,NaN,60948,0.40
1949-09-30,6.6,23.75,NaN,NaN,14182.0,NaN,61633,0.40
1949-12-31,6.6,23.61,NaN,NaN,13999.0,NaN,61908,0.40
1950-03-31,6.3,23.64,NaN,NaN,14490.0,NaN,61604,0.75
